# 1.2 Introduction Statistical Inference #

When a random variable $X$ has a known distribution function $F(x)$ we can find the quantiles of $X$ by inverting this function. For example, the lower $0.05$ quantile of $X$ is given by $F^{-1}(0.05)$, which is a point that cuts off the lower $5\%$ of the distribution.

$$
P(X<F^{-1}(\alpha)) = \alpha.
$$

When $X$ is a sampling random variable, the upper and lower percentiles of $X$ are called its critical values. For a standard normal distribution symmetric about zero, this is easily calculated on python. This is also the inverse of the cumulative distribution function

In [34]:
import scipy.stats
import numpy as np


In [35]:
a = scipy.stats.norm.ppf(0.995)
print(a)
print(scipy.stats.norm.cdf(a))

2.5758293035489004
0.995


A confidence interval is a range where one is confident the true parameter lies to a certain extent. So, $(A,B)$ is a $95\%$ confidence interval for $\theta$ if $P(A<\theta <B) = 0.95$. These confidence intervals with an upper and lower parameter are known as two-sided confidence intervals. We can also consider one-sided confidence intervals, where one of the parameters is $\infty$ or $-\infty$. 

#### Central Limit Theorem ####

Suppose we have a sequence of independent and identically distributed random variables $\{X_1,\dots,X_n\}$ with finite expectation $\mu$ and finite variance $\sigma^2$. The expectation value of the mean is $E(\bar{X}_n)=\mu$, and variance $V(\bar{X}_n) = n^{-1}\sigma^2$. The standard error of the sample mean is $\sigma / \sqrt{n}$. We consider what is the shape of the distribution of the sample mean. Since normal distribution are stable, the sum of $n$ independent normal random variables is another normal variable. 

In the cases where the underlying population is not normally distributed, the Central Limit Theorem states that the distribution of the sample mean will converge to a normal distribution as $n\rightarrow \infty$. Put another way,

$$
\lim_{n \rightarrow \infty} P \left( \frac{\bar{X}_n - \mu}{\sigma / \sqrt{n}} < z \right) = \Phi(z),
$$

where $\Phi(z)$ is the value of the standard normal distribution at $z$. This can be used to help us construct approximate two-sided confidence intervals for an unknown population mean. 

$$
P(\bar{X}_n - \Phi^{-1}(\frac{1+\alpha}{2})\times \frac{\sigma}{\sqrt{n}} < \mu < \bar{X}_n + \Phi^{-1}(\frac{1+\alpha}{2})\times \frac{\sigma}{\sqrt{n}}  ) = \alpha.
$$

However, this does not help us estimate $\sigma$, which is needed in the calculations of $\Phi^{-1}$.

#### Confidence intervals based on Student t Distribution ####

We replace the true variance $\sigma$ with an estimated variance. This causes $\bar{X}_n$ to become t distributed. We thus replace

$$
P \left( \frac{\bar{X}_n - \mu}{\sigma / \sqrt{n}} < x \right) \approx t_{n-1}(x),
$$

where $t_{n-1}(x)$ is the value of the standard Student $t$ distribution with $n-1$ degrees of freedom. This then allows us to construct the approximate confidence interval.


In [36]:
# Function to return mean, and confidence interval
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# Random Data between -100 and 100
data = (np.random.rand(10000)-0.5)*200

print(mean_confidence_interval(data))

(-0.7933779732323292, -1.9208985718108291, 0.3341426253461708)


#### Confidence intervals for Variance ####

Confidence intervals for variance $\sigma^2$ are constructed in a similar way, but using the chi-squared distribution instead

$$
\frac{(n-1)s^2}{\sigma^2} \sim \chi_{n-1}^2.
$$

In [37]:
from scipy.stats.distributions import chi2

# Function to return variance, and confidence interval
def var_confidence_interval(data,confidence=0.95):
    samplevar = scipy.stats.moment(data,moment=2)/(len(data)-1)
    n = len(data)
    alpha = 1-confidence
    df = n - 1 
    lower = (n - 1) * samplevar / chi2.ppf(1 - alpha / 2, df)
    upper = (n - 1) * samplevar / chi2.ppf(alpha / 2, df)
    return samplevar, lower,upper

# Random Data between -100 and 100
data = (np.random.rand(10000)-0.5)*200

print(var_confidence_interval(data))    

(0.336296988922984, 0.32716629365160477, 0.3458171728999795)


#### Hypothesis Tests ####

The framework is:

1) Set up the null $H_0$ and alternative $H_1$ hypotheses.

2) State the test statistic $X$.

3) Choose a significance level.

4) Determine the critical region. This will depend on the test statistic $X$ and the chosen significance level.

5) Evaluate the test statistic.

6) Apply the decision rule.

We illustrate this with a few examples.

### Test on Means ###

Assume that we draw a sample from a population with a normal distribution, but we do not know the expectation nor variance of the distribution. We wish to test $H_0:\mu = 0$ versus $H_1:\mu<0$. The test statistic is:

$$
t = \frac{\bar{X}_n - \mu}{s} \sim t_{n-1}.
$$

Suppose we take a sample of size 10, observe a mean 2.5, and a variance $s^2 = 2.5$. Then the value of the test statistic under the null hypothesis is $5$. The one-sided critical region for a test of size $0.05$ is $(-\infty,-1.83)$. Thus, in this scenario, we cannot reject the null hypothesis in favor of the one-sided alternative $\mu<0$. However, we could reject the null in favour of the alternative $\mu>0$ at a very high level of significance.

Since the t distribution converges to a standard normal distribution if the sample is large, we use the normal distribution for the test, and this is called a Z test.

Now suppose we have two samples, and we want to decide whether they are both drawn from the same population. We can test for the equality of two means using the t statistic:

$$
\frac{\bar{X}_1 - \bar{X}_2}{\sqrt{s_1^2/n_1 + s_2^2/n_2}} \sim t_{n_1 + n_2 -2}.
$$

### Test on Variances ###

Similar methods can be used to test for variances. However, the test statistic $X$ now has a chi-squared distribution with $n-1$ degrees of freedom.

### Maximum Likelihood Estimation ###

MLE is a general method for estimating the parameters of a distribution. It is used extensively as the estimators are consistent, meaning the distribution converges to the true parameter as the sample size increases. The likelihood of a sample $\vec{x} = (x_1,\dots,x_n)$ is the product of the values of the density function at each observation $x_i$. The likelihood is given as

$$
L(\vec{\theta} | \vec{x}) = \prod_{i}^n f(x_i,\vec{\theta}).
$$

Usually, we do not optimize this directly. Instead, we work with the log of the likelihood function. This is valid as log is a monotonic increasing function.

As an example of how this might work, we take a probability density function $\phi$ with expectation $\mu$ and $\sigma$.

$$
\phi(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \left( - \frac{(x-\mu)^2}{2 \sigma^2}  \right) \\
-2 \ln \phi(x) = \text{constant} + \ln(\sigma^2) + \frac{(x-\mu)^2}{\sigma^2}.
$$

Now, if we are given a sample $\vec{x} = (x_1,\dots,x_n)$, maximizing the likelihood is equivalent to minimizing $-2$ times the log likelihood, which is minimizing

$$
-2 \sum_i^n \ln \phi(x_i).
$$

If we differentiate this with respect to $\mu$ and $\sigma^2$, we obtain the conditions

$$
\sum_i^n (x_i - \mu) = 0\\
\sum_i^n (x_i - \mu)^2 = n \sigma^2.
$$

This gives you the estimators

$$
\hat{\mu}=\bar{x} = \frac{1}{n} \sum_i^n x_i \\
\hat{\sigma}^2 = \frac{1}{n} \sum_i^n (x_i-\bar{x})^2.
$$

If you consider the Hessian matrix of second order partial derivatives, we realize it is positive definite, so the stationary point found is a local minimum.

One can also calcualte the standard errors of the maximum likelihood estimates. The information matrix $I(\sigma)$ is the matrix of expected values of the negative of the second derivatives of the log likelihood. The inverse of the information matrix is the covariance matrix of the maximum likelihood estimator. From the diagonals of the matrix, you can then obtain the standard errors of the maximum likelihood estimators.